# Red-Yellow-Comparison-Script

### Libraries

In [15]:
import pandas as pd
import numpy as np
import os
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import geopandas as gpd
import pandas as pd
import fiona
import pprint
from xml.dom.minidom import *
import pandas as pd
import csv
import xml.etree.ElementTree as ET
import re
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import shapely
import math
from zipfile import ZipFile
from tkinter import filedialog
pd.set_option('display.max_columns',None)

### Functions

In [16]:
# Function for Yellow Pre-Estimations
def main_assump_pre_est():
    global df_PreEst
    df_PreEst = pd.read_excel(file, sheet_name='Pre-Estimation')
    template_date = df_PreEst.iloc[2,4]
    template_list= ["Template Date: 08-17-2022-A","Template Date: 08-17-2022-D"]
    if template_date in template_list:
        df_filt_est, control_val = scenario_active()
    else: 
        df_filt_est, control_val = scenario_recommended()
    filt_assp = yellow_assump()
    return filt_assp, df_filt_est, control_val

In [17]:
# Function for those estimate files that were selected based on "Recommended"
def scenario_recommended():
    df_PE_tf = df_PreEst.transpose()
    df_filtered_estimates = df_PE_tf[df_PE_tf.iloc[:,2].str.contains('Recommended', case=False, na=False).transpose()].T
    is_recommended_model = True
    return df_filtered_estimates, is_recommended_model

In [18]:
# Function for those estimate files that were selected based on "Active"
def scenario_active():
    df_PE_tf = df_PreEst.transpose()
    df_filtered_estimates = df_PE_tf[df_PE_tf.iloc[:,3].str.contains('Active', case=True, na=False).transpose()].T
    is_recommended_model = False
    return df_filtered_estimates, is_recommended_model

In [19]:
# Function for Yellow Assumptions
def yellow_assump():
    # Loading the workbook
    wb = load_workbook(filename = file)
    try:
        # Loading the "Assumptions" sheet
        ws = wb["Assumptions"]
    except KeyError:
        # Loading the "Assump" sheet if it is not "Assumptions"
        ws = wb["Assump"]
        # Filtering the first 7 columns and grabbing the columns 8 and after
    filter_list = [False,False,False,False,False,False,False] # we used this because we dont want to return true in 7 columns
    for column in range(8, 100):
        cell = ws.cell(column=column,row=7)
        try :
            if cell.font.color.rgb == "FF0000FF":
                filter_list.append(True)
            else:
                filter_list.append(False)
        except AttributeError:
            filter_list.append(False)
    try:
        # Loading the "Assump" sheet if it is not "Assumptions"
        df_Asp = pd.read_excel(file, sheet_name='Assump')
    except ValueError:
        # Loading the "Assumptions" sheet
        df_Asp = pd.read_excel(file, sheet_name='Assumptions')
    filt_list= filter_list[:df_Asp.shape[1]]
    gen_filt_asp = df_Asp.T[filt_list].T
    # Rejecting the last column next to estimations if it has more than 660 NaN values
    if gen_filt_asp.iloc[:,-1].isna().sum() > 660:
        filt_asp = df_Asp.T[filt_list].T.iloc[:,:-1]
    else:
        filt_asp = df_Asp.T[filt_list].T
    global filter_number
    filter_number = len(filt_asp.columns)
    # Concat the columns before the estimates that contain the variables + the estimates
    # df_excel = pd.concat([yellow_asp_excel,filt_asp],axis=1)
    # df_winston = pd.concat([yellow_asp_winston,filt_asp],axis=1) ##### PROBABLY WILL BE CHANGED
    # return df_excel, df_winston
    return filt_asp

In [20]:
def concat_Est_Assump():
    df_excel = pd.concat([yellow_asp_excel,filter_asum_last],axis=1)
    df_winston = pd.concat([yellow_asp_winston,filter_asum_last],axis=1)
    yellow_p_est_excel1= pd.concat([yellow_p_est_excel,filter_est_last],axis=1)
    df_winston_estimates_w_uid=pd.concat([df_PreEst.iloc[5:,2],filter_est_last.iloc[5:,:]],axis=1)
    yellow_p_est_winston1 = pd.merge(yellow_p_est_winston, 
                      df_winston_estimates_w_uid, 
                      on ='Unnamed: 2', 
                      how ='left')
    return df_excel,df_winston,yellow_p_est_excel1, yellow_p_est_winston1

In [21]:
def scenario_37():
    df_model =pd.read_excel(file, sheet_name="Model")
    df_model1 = df_model.loc[:,"Unnamed: 37":]
    model_estimates =  df_model1.dropna(how='all',axis=1)
    model_temp_df = pd.DataFrame()
    for i in range(len(model_estimates)):
        ds = 6*i
        for m in range(model_estimates.shape[1]):
            x = m//6
            if m % 6 == 0:
                ds = 6*i
            model_temp_df.loc[ds,x] = model_estimates.iloc[i,m]
            ds = ds+1
    model_temp_df = model_temp_df.iloc[:,(-filter_number):]
    last_model_df = pd.concat([model_last_df,model_temp_df],axis=1,ignore_index=True)
    return last_model_df

In [22]:
def scenario_61():
    df_model =pd.read_excel(file, sheet_name="Model")
    df_model1 = df_model.loc[:,"Unnamed: 61":]
    model_estimates =  df_model1.dropna(how='all',axis=1)
    model_temp_df = pd.DataFrame()
    for i in range(len(model_estimates)):
        ds = 6*i
        for m in range(model_estimates.shape[1]):
            x = m//6
            if m % 6 == 0:
                ds = 6*i
            model_temp_df.loc[ds,x] = model_estimates.iloc[i,m]
            ds = ds+1
    model_temp_df = model_temp_df.iloc[:,(-filter_number):]
    last_model_df = pd.concat([model_last_df,model_temp_df],axis=1,ignore_index=True)
    return last_model_df

In [23]:
def creating_model_uid():
    df_model =pd.read_excel(file, sheet_name="Model")
    model_uid = df_model.loc[:,"S_M_0"]
    global model_sub_uid
    model_sub_uid = []
    for i in model_uid:
        for a in range(6):
            model_sub_uid.append((i + "_" + str(a)))
    return

# MAIN CODE

In [36]:
dirname = filedialog.askdirectory()
file_list = os.listdir(dirname)      

In [37]:
uid_column = ['S_ER_6',
 'S_ER_7',
 'S_ER_8',
 'S_ER_9',
 'S_ER_274',
 'S_ER_10',
 'S_ER_11',
 'S_ER_12',
 'S_ER_13',
 'S_ER_14',
 'S_ER_15',
 'S_ER_16',
 'S_ER_17',
 'S_ER_200',
 'S_ER_18',
 'S_ER_19',
 'S_ER_20',
 'S_ER_21',
 'S_ER_22',
 'S_ER_23',
 'S_ER_24',
 'S_ER_25',
 'S_ER_26',
 'S_ER_27',
 'S_ER_28',
 'S_ER_29',
 'S_ER_30',
 'S_ER_31',
 'S_ER_32',
 'S_ER_33',
 'S_ER_34',
 'S_ER_35',
 'S_ER_36',
 'S_ER_37',
 'S_ER_38',
 'S_ER_39',
 'S_ER_40',
 'S_ER_106',
 'S_ER_107',
 'S_ER_203',
 'S_ER_204',
 'S_ER_110',
 'S_ER_113',
 'S_ER_205',
 'S_ER_206',
 'S_ER_118',
 'S_ER_207',
 'S_ER_208',
 'S_ER_209',
 'S_ER_210',
 'S_ER_123',
 'S_ER_125',
 'S_ER_211',
 'S_ER_126',
 'S_ER_127',
 'S_ER_128',
 'S_ER_212',
 'S_ER_293',
 'S_ER_317',
 'S_ER_213',
 'S_ER_214',
 'S_ER_215',
 'S_ER_108',
 'S_ER_111',
 'S_ER_129',
 'S_ER_112',
 'S_ER_216',
 'S_ER_217',
 'S_ER_218',
 'S_ER_219',
 'S_ER_220',
 'S_ER_221',
 'S_ER_222',
 'S_ER_223',
 'S_ER_224',
 'S_ER_225',
 'S_ER_226',
 'S_ER_227',
 'S_ER_228',
 'S_ER_229',
 'S_ER_230',
 'S_ER_231',
 'S_ER_131',
 'S_ER_232',
 'S_ER_233',
 'S_ER_234',
 'S_ER_235',
 'S_ER_236',
 'S_ER_132',
 'S_ER_237',
 'S_ER_1',
 'S_ER_133',
 'S_ER_134',
 'S_ER_197',
 'S_ER_62',
 'S_ER_63',
 'S_ER_360',
 'S_ER_361',
 'S_ER_64',
 'S_ER_61',
 'S_ER_60',
 'S_ER_65',
 'S_ER_66',
 'S_ER_67',
 'S_ER_68',
 'S_ER_69',
 'S_ER_70',
 'S_ER_71',
 'S_ER_72',
 'S_ER_73',
 'S_ER_41',
 'S_ER_42',
 'S_ER_43',
 'S_ER_44',
 'S_ER_45',
 'S_ER_46',
 'S_ER_47',
 'S_ER_48',
 'S_ER_49',
 'S_ER_320',
 'S_ER_321',
 'S_ER_322',
 'S_ER_50',
 'S_ER_51',
 'S_ER_52',
 'S_ER_323',
 'S_ER_356',
 'S_ER_324',
 'S_ER_325',
 'S_ER_326',
 'S_ER_327',
 'S_ER_328',
 'S_ER_329',
 'S_ER_330',
 'S_ER_331',
 'S_ER_357',
 'S_ER_332',
 'S_ER_333',
 'S_ER_334',
 'S_ER_335',
 'S_ER_336',
 'S_ER_337',
 'S_ER_338',
 'S_ER_339',
 'S_ER_358',
 'S_ER_340',
 'S_ER_341',
 'S_ER_342',
 'S_ER_343',
 'S_ER_344',
 'S_ER_53',
 'S_ER_345',
 'S_ER_346',
 'S_ER_347',
 'S_ER_54',
 'S_ER_55',
 'S_ER_56',
 'S_ER_75',
 'S_ER_57',
 'S_ER_58',
 'S_ER_319',
 'S_ER_277',
 'S_ER_278',
 'S_ER_279',
 'S_ER_280',
 'S_ER_281',
 'S_ER_275',
 'S_ER_276',
 'S_ER_359',
 'S_ER_80',
 'S_ER_81',
 'S_ER_348',
 'S_ER_83',
 'S_ER_349',
 'S_ER_84',
 'S_ER_283',
 'S_ER_284',
 'S_ER_85',
 'S_ER_285',
 'S_ER_88',
 'S_ER_89',
 'S_ER_90',
 'S_ER_286',
 'S_ER_201',
 'S_ER_202',
 'S_ER_198',
 'S_ER_289',
 'S_ER_74',
 'S_ER_199',
 'S_ER_91',
 'S_ER_92',
 'S_ER_93',
 'S_ER_94',
 'S_ER_95',
 'S_ER_290',
 'S_ER_291',
 'S_ER_292',
 'S_ER_96',
 'S_ER_97',
 'S_ER_98',
 'S_ER_99',
 'S_ER_100',
 'S_ER_101',
 'S_ER_102',
 'S_ER_103',
 'S_ER_104',
 'S_ER_135',
 'S_ER_136',
 'S_ER_294',
 'S_ER_137',
 'S_ER_138',
 'S_ER_139',
 'S_ER_140',
 'S_ER_141',
 'S_ER_142',
 'S_ER_143',
 'S_ER_318',
 'S_ER_144',
 'S_ER_145',
 'S_ER_146',
 'S_ER_148',
 'S_ER_149',
 'S_ER_150',
 'S_ER_151',
 'S_ER_152',
 'S_ER_153',
 'S_ER_154',
 'S_ER_156',
 'S_ER_157',
 'S_ER_158',
 'S_ER_159',
 'S_ER_160',
 'S_ER_161',
 'S_ER_295',
 'S_ER_296',
 'S_ER_163',
 'S_ER_164',
 'S_ER_165',
 'S_ER_166',
 'S_ER_167',
 'S_ER_168',
 'S_ER_169',
 'S_ER_170',
 'S_ER_171',
 'S_ER_172',
 'S_ER_173',
 'S_ER_174',
 'S_ER_175',
 'S_ER_176',
 'S_ER_177',
 'S_ER_178',
 'S_ER_179',
 'S_ER_180',
 'S_ER_181',
 'S_ER_182',
 'S_ER_183',
 'S_ER_184',
 'S_ER_185',
 'S_ER_186',
 'S_ER_187',
 'S_ER_188',
 'S_ER_189',
 'S_ER_190',
 'S_ER_191',
 'S_ER_192',
 'S_ER_193',
 'S_ER_194',
 'S_ER_195',
 'S_ER_196',
 'S_ER_273',
 'S_ER_297',
 'S_ER_298',
 'S_ER_238',
 'S_ER_239',
 'S_ER_240',
 'S_ER_241',
 'S_ER_242',
 'S_ER_243',
 'S_ER_244',
 'S_ER_245',
 'S_ER_246',
 'S_ER_247',
 'S_ER_248',
 'S_ER_249',
 'S_ER_250',
 'S_ER_251',
 'S_ER_252',
 'S_ER_253',
 'S_ER_254',
 'S_ER_255',
 'S_ER_256',
 'S_ER_257',
 'S_ER_258',
 'S_ER_259',
 'S_ER_260',
 'S_ER_261',
 'S_ER_262',
 'S_ER_263',
 'S_ER_264',
 'S_ER_265',
 'S_ER_266',
 'S_ER_267',
 'S_ER_268',
 'S_ER_269',
 'S_ER_270',
 'S_ER_271',
 'S_ER_272',
 'S_ER_162',
 'S_ER_155',
 'S_ER_130',
 'S_ER_124',
 'S_ER_122',
 'S_ER_2',
 'S_ER_3',
 'S_ER_4',
 'S_ER_5',
 'S_ER_59',
 'S_ER_76',
 'S_ER_77',
 'S_ER_78',
 'S_ER_79',
 'S_ER_82',
 'S_ER_86',
 'S_ER_87',
 'S_ER_105',
 'S_ER_109',
 'S_ER_114',
 'S_ER_115',
 'S_ER_116',
 'S_ER_117',
 'S_ER_119',
 'S_ER_120',
 'S_ER_121',
 'S_ER_299',
 'S_ER_300',
 'S_ER_301',
 'S_ER_302',
 'S_ER_303',
 'S_ER_304',
 'S_ER_305',
 'S_ER_306',
 'S_ER_307',
 'S_ER_308',
 'S_ER_309',
 'S_ER_310',
 'S_ER_311',
 'S_ER_312',
 'S_ER_313',
 'S_ER_314',
 'S_ER_315',
 'S_ER_316',
 'S_ER_350',
 'S_ER_351',
 'S_ER_352',
 'S_ER_353',
 'S_ER_354',
 'S_ER_355']

In [38]:
asp_uid_list = ['S_A_1',
 'S_A_2',
 'S_A_3',
 'S_A_4',
 'S_A_5',
 'S_A_735',
 'S_A_736',
 'S_A_742',
 'S_A_6',
 'S_A_7',
 'S_A_8',
 'S_A_9',
 'S_A_10',
 'S_A_11',
 'S_A_12',
 'S_A_737',
 'S_A_13',
 'S_A_14',
 'S_A_15',
 'S_A_16',
 'S_A_17',
 'S_A_18',
 'S_A_19',
 'S_A_20',
 'S_A_21',
 'S_A_22',
 'S_A_23',
 'S_A_24',
 'S_A_25',
 'S_A_26',
 'S_A_27',
 'S_A_28',
 'S_A_29',
 'S_A_30',
 'S_A_31',
 'S_A_32',
 'S_A_33',
 'S_A_38',
 'S_A_39',
 'S_A_40',
 'S_A_37',
 'S_A_45',
 'S_A_46',
 'S_A_41',
 'S_A_42',
 'S_A_43',
 'S_A_44',
 'S_A_34',
 'S_A_35',
 'S_A_36',
 'S_A_755',
 'S_A_743',
 'S_A_744',
 'S_A_745',
 'S_A_746',
 'S_A_753',
 'S_A_47',
 'S_A_48',
 'S_A_756',
 'S_A_740',
 'S_A_53',
 'S_A_738',
 'S_A_54',
 'S_A_57',
 'S_A_58',
 'S_A_59',
 'S_A_60',
 'S_A_61',
 'S_A_62',
 'S_A_63',
 'S_A_64',
 'S_A_65',
 'S_A_748',
 'S_A_66',
 'S_A_67',
 'S_A_68',
 'S_A_69',
 'S_A_70',
 'S_A_71',
 'S_A_72',
 'S_A_73',
 'S_A_74',
 'S_A_75',
 'S_A_76',
 'S_A_77',
 'S_A_78',
 'S_A_79',
 'S_A_80',
 'S_A_81',
 'S_A_82',
 'S_A_84',
 'S_A_90',
 'S_A_91',
 'S_A_92',
 'S_A_93',
 'S_A_94',
 'S_A_97',
 'S_A_99',
 'S_A_100',
 'S_A_101',
 'S_A_104',
 'S_A_106',
 'S_A_107',
 'S_A_108',
 'S_A_109',
 'S_A_110',
 'S_A_111',
 'S_A_741',
 'S_A_112',
 'S_A_113',
 'S_A_114',
 'S_A_115',
 'S_A_116',
 'S_A_117',
 'S_A_118',
 'S_A_119',
 'S_A_120',
 'S_A_121',
 'S_A_122',
 'S_A_123',
 'S_A_124',
 'S_A_125',
 'S_A_126',
 'S_A_127',
 'S_A_128',
 'S_A_129',
 'S_A_130',
 'S_A_131',
 'S_A_132',
 'S_A_133',
 'S_A_134',
 'S_A_135',
 'S_A_136',
 'S_A_137',
 'S_A_138',
 'S_A_139',
 'S_A_140',
 'S_A_141',
 'S_A_142',
 'S_A_143',
 'S_A_144',
 'S_A_145',
 'S_A_146',
 'S_A_147',
 'S_A_148',
 'S_A_149',
 'S_A_150',
 'S_A_151',
 'S_A_152',
 'S_A_153',
 'S_A_154',
 'S_A_155',
 'S_A_156',
 'S_A_157',
 'S_A_158',
 'S_A_159',
 'S_A_160',
 'S_A_161',
 'S_A_162',
 'S_A_163',
 'S_A_164',
 'S_A_165',
 'S_A_166',
 'S_A_167',
 'S_A_168',
 'S_A_169',
 'S_A_170',
 'S_A_171',
 'S_A_172',
 'S_A_173',
 'S_A_174',
 'S_A_175',
 'S_A_176',
 'S_A_177',
 'S_A_178',
 'S_A_179',
 'S_A_180',
 'S_A_181',
 'S_A_182',
 'S_A_183',
 'S_A_184',
 'S_A_185',
 'S_A_186',
 'S_A_187',
 'S_A_188',
 'S_A_189',
 'S_A_190',
 'S_A_191',
 'S_A_192',
 'S_A_193',
 'S_A_194',
 'S_A_195',
 'S_A_196',
 'S_A_197',
 'S_A_198',
 'S_A_199',
 'S_A_200',
 'S_A_201',
 'S_A_202',
 'S_A_203',
 'S_A_204',
 'S_A_754',
 'S_A_205',
 'S_A_206',
 'S_A_207',
 'S_A_208',
 'S_A_209',
 'S_A_210',
 'S_A_211',
 'S_A_212',
 'S_A_213',
 'S_A_214',
 'S_A_215',
 'S_A_216',
 'S_A_217',
 'S_A_218',
 'S_A_219',
 'S_A_238',
 'S_A_239',
 'S_A_240',
 'S_A_241',
 'S_A_242',
 'S_A_243',
 'S_A_244',
 'S_A_245',
 'S_A_246',
 'S_A_247',
 'S_A_248',
 'S_A_249',
 'S_A_250',
 'S_A_251',
 'S_A_252',
 'S_A_253',
 'S_A_254',
 'S_A_255',
 'S_A_256',
 'S_A_257',
 'S_A_258',
 'S_A_259',
 'S_A_260',
 'S_A_261',
 'S_A_262',
 'S_A_263',
 'S_A_264',
 'S_A_265',
 'S_A_266',
 'S_A_267',
 'S_A_268',
 'S_A_269',
 'S_A_270',
 'S_A_271',
 'S_A_272',
 'S_A_273',
 'S_A_274',
 'S_A_275',
 'S_A_276',
 'S_A_277',
 'S_A_278',
 'S_A_279',
 'S_A_280',
 'S_A_281',
 'S_A_282',
 'S_A_283',
 'S_A_284',
 'S_A_285',
 'S_A_286',
 'S_A_751',
 'S_A_752',
 'S_A_287',
 'S_A_288',
 'S_A_290',
 'S_A_291',
 'S_A_292',
 'S_A_293',
 'S_A_295',
 'S_A_296',
 'S_A_297',
 'S_A_298',
 'S_A_299',
 'S_A_300',
 'S_A_301',
 'S_A_302',
 'S_A_303',
 'S_A_304',
 'S_A_305',
 'S_A_306',
 'S_A_307',
 'S_A_308',
 'S_A_309',
 'S_A_310',
 'S_A_311',
 'S_A_312',
 'S_A_313',
 'S_A_314',
 'S_A_315',
 'S_A_316',
 'S_A_317',
 'S_A_318',
 'S_A_319',
 'S_A_320',
 'S_A_321',
 'S_A_322',
 'S_A_323',
 'S_A_324',
 'S_A_325',
 'S_A_326',
 'S_A_327',
 'S_A_328',
 'S_A_329',
 'S_A_330',
 'S_A_331',
 'S_A_332',
 'S_A_333',
 'S_A_334',
 'S_A_335',
 'S_A_336',
 'S_A_337',
 'S_A_338',
 'S_A_339',
 'S_A_340',
 'S_A_341',
 'S_A_342',
 'S_A_343',
 'S_A_344',
 'S_A_345',
 'S_A_346',
 'S_A_347',
 'S_A_348',
 'S_A_349',
 'S_A_350',
 'S_A_351',
 'S_A_352',
 'S_A_353',
 'S_A_354',
 'S_A_355',
 'S_A_356',
 'S_A_357',
 'S_A_358',
 'S_A_359',
 'S_A_360',
 'S_A_361',
 'S_A_362',
 'S_A_363',
 'S_A_364',
 'S_A_365',
 'S_A_366',
 'S_A_367',
 'S_A_368',
 'S_A_369',
 'S_A_370',
 'S_A_371',
 'S_A_372',
 'S_A_373',
 'S_A_374',
 'S_A_375',
 'S_A_376',
 'S_A_377',
 'S_A_378',
 'S_A_379',
 'S_A_380',
 'S_A_381',
 'S_A_382',
 'S_A_383',
 'S_A_384',
 'S_A_385',
 'S_A_386',
 'S_A_387',
 'S_A_388',
 'S_A_389',
 'S_A_390',
 'S_A_391',
 'S_A_392',
 'S_A_393',
 'S_A_394',
 'S_A_395',
 'S_A_396',
 'S_A_397',
 'S_A_398',
 'S_A_399',
 'S_A_400',
 'S_A_401',
 'S_A_402',
 'S_A_403',
 'S_A_404',
 'S_A_405',
 'S_A_406',
 'S_A_407',
 'S_A_408',
 'S_A_409',
 'S_A_410',
 'S_A_411',
 'S_A_412',
 'S_A_413',
 'S_A_414',
 'S_A_415',
 'S_A_416',
 'S_A_417',
 'S_A_418',
 'S_A_419',
 'S_A_420',
 'S_A_421',
 'S_A_422',
 'S_A_423',
 'S_A_424',
 'S_A_425',
 'S_A_426',
 'S_A_427',
 'S_A_428',
 'S_A_429',
 'S_A_430',
 'S_A_431',
 'S_A_432',
 'S_A_433',
 'S_A_434',
 'S_A_435',
 'S_A_436',
 'S_A_437',
 'S_A_438',
 'S_A_439',
 'S_A_440',
 'S_A_441',
 'S_A_442',
 'S_A_443',
 'S_A_444',
 'S_A_445',
 'S_A_446',
 'S_A_447',
 'S_A_448',
 'S_A_449',
 'S_A_450',
 'S_A_451',
 'S_A_452',
 'S_A_453',
 'S_A_454',
 'S_A_455',
 'S_A_456',
 'S_A_457',
 'S_A_458',
 'S_A_459',
 'S_A_460',
 'S_A_461',
 'S_A_462',
 'S_A_463',
 'S_A_464',
 'S_A_465',
 'S_A_466',
 'S_A_467',
 'S_A_468',
 'S_A_469',
 'S_A_470',
 'S_A_471',
 'S_A_472',
 'S_A_473',
 'S_A_474',
 'S_A_475',
 'S_A_476',
 'S_A_477',
 'S_A_478',
 'S_A_479',
 'S_A_480',
 'S_A_481',
 'S_A_482',
 'S_A_483',
 'S_A_484',
 'S_A_499',
 'S_A_500',
 'S_A_501',
 'S_A_502',
 'S_A_503',
 'S_A_504',
 'S_A_511',
 'S_A_512',
 'S_A_513',
 'S_A_514',
 'S_A_515',
 'S_A_516',
 'S_A_517',
 'S_A_518',
 'S_A_519',
 'S_A_520',
 'S_A_521',
 'S_A_522',
 'S_A_523',
 'S_A_524',
 'S_A_525',
 'S_A_526',
 'S_A_527',
 'S_A_528',
 'S_A_529',
 'S_A_530',
 'S_A_721',
 'S_A_531',
 'S_A_532',
 'S_A_533',
 'S_A_534',
 'S_A_535',
 'S_A_536',
 'S_A_722',
 'S_A_537',
 'S_A_538',
 'S_A_539',
 'S_A_540',
 'S_A_541',
 'S_A_542',
 'S_A_543',
 'S_A_544',
 'S_A_545',
 'S_A_546',
 'S_A_547',
 'S_A_548',
 'S_A_549',
 'S_A_550',
 'S_A_551',
 'S_A_552',
 'S_A_553',
 'S_A_554',
 'S_A_555',
 'S_A_556',
 'S_A_557',
 'S_A_558',
 'S_A_559',
 'S_A_560',
 'S_A_561',
 'S_A_562',
 'S_A_563',
 'S_A_564',
 'S_A_565',
 'S_A_566',
 'S_A_567',
 'S_A_568',
 'S_A_569',
 'S_A_570',
 'S_A_571',
 'S_A_572',
 'S_A_573',
 'S_A_574',
 'S_A_575',
 'S_A_576',
 'S_A_577',
 'S_A_578',
 'S_A_579',
 'S_A_580',
 'S_A_581',
 'S_A_582',
 'S_A_583',
 'S_A_584',
 'S_A_585',
 'S_A_586',
 'S_A_587',
 'S_A_588',
 'S_A_589',
 'S_A_590',
 'S_A_591',
 'S_A_592',
 'S_A_593',
 'S_A_594',
 'S_A_595',
 'S_A_596',
 'S_A_597',
 'S_A_598',
 'S_A_599',
 'S_A_600',
 'S_A_601',
 'S_A_602',
 'S_A_603',
 'S_A_604',
 'S_A_605',
 'S_A_606',
 'S_A_607',
 'S_A_608',
 'S_A_609',
 'S_A_610',
 'S_A_611',
 'S_A_612',
 'S_A_613',
 'S_A_614',
 'S_A_615',
 'S_A_616',
 'S_A_617',
 'S_A_618',
 'S_A_619',
 'S_A_620',
 'S_A_621',
 'S_A_622',
 'S_A_623',
 'S_A_624',
 'S_A_625',
 'S_A_626',
 'S_A_627',
 'S_A_628',
 'S_A_629',
 'S_A_630',
 'S_A_631',
 'S_A_632',
 'S_A_633',
 'S_A_634',
 'S_A_635',
 'S_A_636',
 'S_A_637',
 'S_A_638',
 'S_A_639',
 'S_A_640',
 'S_A_641',
 'S_A_642',
 'S_A_643',
 'S_A_644',
 'S_A_645',
 'S_A_646',
 'S_A_647',
 'S_A_648',
 'S_A_649',
 'S_A_650',
 'S_A_651',
 'S_A_652',
 'S_A_653',
 'S_A_654',
 'S_A_655',
 'S_A_656',
 'S_A_657',
 'S_A_658',
 'S_A_727',
 'S_A_728',
 'S_A_729',
 'S_A_730',
 'S_A_731',
 'S_A_732',
 'S_A_733',
 'S_A_734',
 'S_A_659',
 'S_A_660',
 'S_A_661',
 'S_A_662',
 'S_A_663',
 'S_A_664',
 'S_A_665',
 'S_A_666',
 'S_A_667',
 'S_A_668',
 'S_A_669',
 'S_A_670',
 'S_A_671',
 'S_A_672',
 'S_A_723',
 'S_A_674',
 'S_A_675',
 'S_A_676',
 'S_A_677',
 'S_A_678',
 'S_A_679',
 'S_A_680',
 'S_A_681',
 'S_A_682',
 'S_A_683',
 'S_A_724',
 'S_A_685',
 'S_A_686',
 'S_A_687',
 'S_A_688',
 'S_A_689',
 'S_A_690',
 'S_A_691',
 'S_A_692',
 'S_A_693',
 'S_A_694',
 'S_A_725',
 'S_A_696',
 'S_A_697',
 'S_A_698',
 'S_A_699',
 'S_A_700',
 'S_A_701',
 'S_A_702',
 'S_A_703',
 'S_A_704',
 'S_A_705',
 'S_A_726',
 'S_A_707',
 'S_A_708',
 'S_A_709',
 'S_A_710',
 'S_A_711',
 'S_A_712',
 'S_A_713',
 'S_A_714',
 'S_A_715',
 'S_A_716',
 'S_A_717',
 'S_A_718',
 'S_A_719',
 'S_A_720']

In [39]:
yellow_p_est_excel = pd.DataFrame()
yellow_p_est_winston = pd.DataFrame({"Unnamed: 2":uid_column})
yellow_asp_excel = pd.DataFrame()
yellow_asp_winston = pd.DataFrame()
model_last_df= pd.DataFrame()

In [40]:
for i in range(len(file_list)):
    file = dirname+"/"+file_list[i]
    filter_asum_last, filter_est_last, controlling_value = main_assump_pre_est()
    if len(filter_asum_last.columns) == len(filter_est_last.columns):
        yellow_asp_excel,yellow_asp_winston,yellow_p_est_excel,yellow_p_est_winston = concat_Est_Assump() 
        if controlling_value == True:
            model_last_df= scenario_37()
        else:
            model_last_df = scenario_61()
    else:
        print(file_list[i])
    if i == 0:
        creating_model_uid()
model_last_df.set_index(pd.Series(model_sub_uid),inplace=True)
yellow_asp_winston.set_index(pd.Series(asp_uid_list), inplace=True)
yellow_p_est_winston.set_index("Unnamed: 2",drop=True,inplace=True)
yellow_asp_winston.columns = model_last_df.columns
yellow_p_est_winston.columns = model_last_df.columns
yellow_winstom_dump = pd.concat([yellow_p_est_winston,yellow_asp_winston,model_last_df],axis=0)

# if len(current_index) > len(new_excel_file_index)
# curent_index - new_idnex columns to merge them to not make a mistake in matching


C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\O

MNBentonSolarEstimate20220907Distributed_906921314.xlsx


C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\OXK0A0A\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
